In [1]:
from __future__ import print_function
import tensorflow as tf
#from vgg import VGG


#from data_loader import Loader

        

import keras
from keras.datasets import cifar100
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras import optimizers
import numpy as np
from keras.layers.core import Lambda
from keras import backend as K
from keras import regularizers

from keras.utils import np_utils
from keras import datasets

import tensorflow as tf

import matplotlib.pyplot as plt

In [2]:
(X_train, Y_train), (X_test, Y_test) = datasets.cifar100.load_data()

# print('label : ',Y_train[0])
# plt.imshow(X_train[0])

Y_train = np_utils.to_categorical(Y_train)
Y_test = np_utils.to_categorical(Y_test)

print(X_train.shape, Y_train.shape)


class cifar100vgg:
    def __init__(self,train=True):
        self.num_classes = 100
        self.weight_decay = 0.0005
        self.x_shape = [32,32,3]

        self.model = self.build_model()
        if train:
            self.model = self.train(self.model)
        else:
            self.model.load_weights('cifar100vgg.h5')


    def build_model(self):
        # Build the network of vgg for 10 classes with massive dropout and weight decay as described in the paper.
        with tf.device("/gpu:0"):

                model = Sequential()
                weight_decay = self.weight_decay

                model.add(Conv2D(64, (3, 3), padding='same',
                                 input_shape=self.x_shape,kernel_regularizer=regularizers.l2(weight_decay)))
                model.add(Activation('relu'))
                model.add(BatchNormalization())
                model.add(Dropout(0.3))

                model.add(Conv2D(64, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
                model.add(Activation('relu'))
                model.add(BatchNormalization())

                model.add(MaxPooling2D(pool_size=(2, 2)))

                model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
                model.add(Activation('relu'))
                model.add(BatchNormalization())
                model.add(Dropout(0.4))

                model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
                model.add(Activation('relu'))
                model.add(BatchNormalization())

                model.add(MaxPooling2D(pool_size=(2, 2)))

                model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
                model.add(Activation('relu'))
                model.add(BatchNormalization())
                model.add(Dropout(0.4))

                model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
                model.add(Activation('relu'))
                model.add(BatchNormalization())
                model.add(Dropout(0.4))

                model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
                model.add(Activation('relu'))
                model.add(BatchNormalization())

                model.add(MaxPooling2D(pool_size=(2, 2)))


                model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
                model.add(Activation('relu'))
                model.add(BatchNormalization())
                model.add(Dropout(0.4))

                model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
                model.add(Activation('relu'))
                model.add(BatchNormalization())
                model.add(Dropout(0.4))

                model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
                model.add(Activation('relu'))
                model.add(BatchNormalization())

                model.add(MaxPooling2D(pool_size=(2, 2)))


                model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
                model.add(Activation('relu'))
                model.add(BatchNormalization())
                model.add(Dropout(0.4))

                model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
                model.add(Activation('relu'))
                model.add(BatchNormalization())
                model.add(Dropout(0.4))

                model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
                model.add(Activation('relu'))
                model.add(BatchNormalization())

                model.add(MaxPooling2D(pool_size=(2, 2)))
                model.add(Dropout(0.5))

                model.add(Flatten())
                model.add(Dense(512,kernel_regularizer=regularizers.l2(weight_decay)))
                model.add(Activation('relu'))
                model.add(BatchNormalization())

                model.add(Dropout(0.5))
                model.add(Dense(self.num_classes))
                model.add(Activation('softmax'))
                return model


    def normalize(self,X_train,X_test):
        #this function normalize inputs for zero mean and unit variance
        # it is used when training a model.
        # Input: training set and test set
        # Output: normalized training set and test set according to the trianing set statistics.
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        print(mean)
        print(std)
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test

    def normalize_production(self,x):
        #this function is used to normalize instances in production according to saved training set statistics
        # Input: X - a training set
        # Output X - a normalized training set according to normalization constants.

        #these values produced during first training and are general for the standard cifar10 training set normalization
        mean = 121.936
        std = 68.389
        return (x-mean)/(std+1e-7)

    def predict(self,x,normalize=True,batch_size=50):
        if normalize:
            x = self.normalize_production(x)
        return self.model.predict(x,batch_size)

    def train(self,model):

        #training parameters
        batch_size = 256
        maxepoches = 200
        learning_rate = 0.1
        lr_decay = 1e-6
        lr_drop = 20

        # The data, shuffled and split between train and test sets:
        (x_train, y_train), (x_test, y_test) = cifar100.load_data()
        x_train = x_train.astype('float32')
        x_test = x_test.astype('float32')
        x_train, x_test = self.normalize(x_train, x_test)

        y_train = keras.utils.to_categorical(y_train, self.num_classes)
        y_test = keras.utils.to_categorical(y_test, self.num_classes)


        def lr_scheduler(epoch):
            return learning_rate * (0.5 ** (epoch // lr_drop))
        reduce_lr = keras.callbacks.LearningRateScheduler(lr_scheduler)


        #data augmentation
        datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False)  # randomly flip images
        # (std, mean, and principal components if ZCA whitening is applied).
        datagen.fit(x_train)



        #optimization details
        sgd = optimizers.SGD(lr=learning_rate, decay=lr_decay, momentum=0.9, nesterov=True)
        model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])


        # training process in a for loop with learning rate drop every 25 epoches.

        historytemp = model.fit_generator(datagen.flow(x_train, y_train,
                                         batch_size=batch_size),
                            steps_per_epoch=x_train.shape[0] // batch_size,
                            epochs=maxepoches,
                            validation_data=(x_test, y_test),callbacks=[reduce_lr],verbose=2)
        model.save_weights('cifar100vgg.h5')
        return model

(50000, 32, 32, 3) (50000, 100)


In [3]:
model = cifar100vgg()

predicted_x = model.predict(X_test)

121.93584
68.38902
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/200
195/195 - 66s - loss: 14.3885 - accuracy: 0.0202 - val_loss: 13.9351 - val_accuracy: 0.0302 - lr: 0.1000
Epoch 2/200
195/195 - 25s - loss: 11.2116 - accuracy: 0.0406 - val_loss: 10.1015 - val_accuracy: 0.0253 - lr: 0.1000
Epoch 3/200
195/195 - 24s - loss: 8.6934 - accuracy: 0.0624 - val_loss: 8.3611 - val_accuracy: 0.0214 - lr: 0.1000
Epoch 4/200
195/195 - 25s - loss: 7.0806 - accuracy: 0.0792 - val_loss: 7.0405 - val_accuracy: 0.0323 - lr: 0.1000
Epoch 5/200
195/195 - 26s - loss: 6.0253 - accuracy: 0.0926 - val_loss: 6.2170 - val_accuracy: 0.0379 - lr: 0.1000
Epoch 6/200
195/195 - 25s - loss: 5.2908 - accuracy: 0.1062 - val_loss: 5.4044 - val_accuracy: 0.0573 - lr: 0.1000
Epoch 7/200
195/195 - 26s - loss: 4.7884 - accuracy: 0.1189 - val_loss: 4.7855 - val_accuracy: 0.0954 - lr: 0.1000
Epoch 8/200
195/195 - 25s - loss: 4.4090 - accuracy: 0.1408 - val_loss: 4.2945 - val_accuracy: 0

Epoch 70/200
195/195 - 27s - loss: 1.9159 - accuracy: 0.6963 - val_loss: 2.2827 - val_accuracy: 0.6357 - lr: 0.0125
Epoch 71/200
195/195 - 27s - loss: 1.9001 - accuracy: 0.6979 - val_loss: 2.2183 - val_accuracy: 0.6418 - lr: 0.0125
Epoch 72/200
195/195 - 27s - loss: 1.9071 - accuracy: 0.6956 - val_loss: 2.2687 - val_accuracy: 0.6343 - lr: 0.0125
Epoch 73/200
195/195 - 27s - loss: 1.8934 - accuracy: 0.7005 - val_loss: 2.1782 - val_accuracy: 0.6484 - lr: 0.0125
Epoch 74/200
195/195 - 27s - loss: 1.8866 - accuracy: 0.7004 - val_loss: 2.3411 - val_accuracy: 0.6188 - lr: 0.0125
Epoch 75/200
195/195 - 27s - loss: 1.8854 - accuracy: 0.7025 - val_loss: 2.3083 - val_accuracy: 0.6290 - lr: 0.0125
Epoch 76/200
195/195 - 27s - loss: 1.8656 - accuracy: 0.7070 - val_loss: 2.2902 - val_accuracy: 0.6318 - lr: 0.0125
Epoch 77/200
195/195 - 27s - loss: 1.8805 - accuracy: 0.7023 - val_loss: 2.2170 - val_accuracy: 0.6437 - lr: 0.0125
Epoch 78/200
195/195 - 27s - loss: 1.8868 - accuracy: 0.7024 - val_loss:

Epoch 141/200
195/195 - 27s - loss: 0.9858 - accuracy: 0.8826 - val_loss: 2.1159 - val_accuracy: 0.6822 - lr: 7.8125e-04
Epoch 142/200
195/195 - 27s - loss: 0.9688 - accuracy: 0.8888 - val_loss: 2.0836 - val_accuracy: 0.6914 - lr: 7.8125e-04
Epoch 143/200
195/195 - 27s - loss: 0.9606 - accuracy: 0.8894 - val_loss: 2.0670 - val_accuracy: 0.6949 - lr: 7.8125e-04
Epoch 144/200
195/195 - 27s - loss: 0.9501 - accuracy: 0.8929 - val_loss: 2.0786 - val_accuracy: 0.6909 - lr: 7.8125e-04
Epoch 145/200
195/195 - 27s - loss: 0.9454 - accuracy: 0.8932 - val_loss: 2.0967 - val_accuracy: 0.6879 - lr: 7.8125e-04
Epoch 146/200
195/195 - 27s - loss: 0.9441 - accuracy: 0.8930 - val_loss: 2.0953 - val_accuracy: 0.6895 - lr: 7.8125e-04
Epoch 147/200
195/195 - 27s - loss: 0.9438 - accuracy: 0.8931 - val_loss: 2.0948 - val_accuracy: 0.6893 - lr: 7.8125e-04
Epoch 148/200
195/195 - 27s - loss: 0.9405 - accuracy: 0.8930 - val_loss: 2.0803 - val_accuracy: 0.6926 - lr: 7.8125e-04
Epoch 149/200
195/195 - 27s - lo

NameError: name 'x_test' is not defined

In [17]:
predicted_x = model.predict(X_test)
residuals = (np.argmax(predicted_x,1)!=np.argmax(Y_test,1))
loss = sum(residuals)/len(residuals)

In [20]:
print("the test acuraccy is:",1-loss)

the test acuraccy is: 0.6960999999999999
